In [1]:
import torch
import transformers
import trl
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [2]:
from huggingface_hub import login
'''
login(
  #token="Your token",  
  add_to_git_credential=True
)
'''

'\nlogin(\n  #token="Your token",  \n  add_to_git_credential=True\n)\n'

In [3]:
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir= '/workspace/gemma-2-9b-it',
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# 데이터

In [2]:
dataset = load_dataset("json", data_files="/workspace/train_dataset.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"].select(range(2000))
test_dataset = dataset["test"].select(range(20))

In [3]:

train_valid= train_dataset.train_test_split(test_size=0.1, seed=45)
train_dataset = train_valid["train"].select(range(20))
valid_dataset = train_valid["test"].select(range(20))

In [4]:
train_dataset

Dataset({
    features: ['messages'],
    num_rows: 20
})

# 학습

In [7]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

args = TrainingArguments(
    output_dir="./model_output", 
    num_train_epochs=1,          
    per_device_train_batch_size=4,
    gradient_accumulation_steps=5,
    gradient_checkpointing=True,  
    optim="adamw_torch_fused",    
    logging_steps=20,            
    eval_strategy="steps",
    eval_steps=20,
    save_steps =20,
    learning_rate=2e-4,           
    bf16=True,                    
    tf32=True,                    
    max_grad_norm=0.3,            
    warmup_ratio=0.03,            
    lr_scheduler_type="constant", 
    #push_to_hub=True,             
    report_to="wandb",            
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
)
trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: wooseoksong (wooseoksong-study). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 d

Step,Training Loss,Validation Loss


TrainOutput(global_step=1, training_loss=2.8632726669311523, metrics={'train_runtime': 56.4729, 'train_samples_per_second': 0.514, 'train_steps_per_second': 0.018, 'total_flos': 564540871802880.0, 'train_loss': 2.8632726669311523, 'epoch': 0.625})